# Introduction to Structured Responses for Computer System Specifications

This script demonstrates how to use **structured responses** with DeepSeek Models (V3 and R1) on Fireworks AI to generate reliable and structured computer system specifications.


### Why Use Structured Responses?
- **Predictability**: Enforces hardware constraints to prevent illogical specifications.
- **Automation**: Structured outputs can be directly used in PC builders, recommendation engines, and procurement systems.
- **Integration**: JSON and Grammar Mode allow seamless integration into applications that require structured PC configurations.

### What You'll Learn
1. **Basic API Setup**: Connecting to Fireworks AI and using DeepSeek v3 and R1.
2. **Grammar Mode**: Generating computer specifications within predefined constraints.
3. **JSON Mode**: Ensuring machine-readable structured responses.
4. **Reasoning JSON Mode**: Capturing the model's thought process alongside structured responses.

### Setup

Install all the dependencies and import the required python modules.

In [45]:
!pip3 install --upgrade fireworks-ai pydantic python-dotenv > /dev/null

### Setup your API Key

In order to use the DeepSeek models, you must first obtain Fireworks API Keys. If you don't already have one, create a [Fireworks AI](https://fireworks.ai) account. Under Account Settings, click on [API Keys](https://fireworks.ai/api-keys) to create one.

In [46]:
from fireworks.client import Fireworks
import os
from dotenv import load_dotenv  # Import dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize Fireworks client using API key from .env
client = Fireworks(api_key=os.getenv("FIREWORKS_API_KEY"))

# 0️⃣ **Basic API Call to DeepSeek v3**

### Chat Completions API
This is a simple test call to ensure the Fireworks API is working correctly.

In [47]:
model_name = "accounts/fireworks/models/deepseek-v3"

response = client.chat.completions.create(
    model=model_name,
    messages=[
        {"role": "user", "content": "What are the latest trends in gaming PC hardware?"},
    ],
)
print(response.choices[0].message.content)

As of 2023, the gaming PC hardware landscape continues to evolve rapidly, driven by advancements in technology and increasing demand for high-performance gaming experiences. Here are some of the latest trends in gaming PC hardware:

### 1. **Next-Gen GPUs (Graphics Processing Units)**
   - **NVIDIA's GeForce RTX 40 Series**: Built on the Ada Lovelace architecture, these GPUs offer significant performance improvements, especially in ray tracing and AI-driven features like DLSS 3.0.
   - **AMD's Radeon RX 7000 Series**: Based on the RDNA 3 architecture, these GPUs provide competitive performance and efficiency, with a focus on 4K gaming and ray tracing capabilities.

### 2. **CPUs (Central Processing Units)**
   - **Intel's 13th Gen Raptor Lake**: Featuring a hybrid architecture with Performance and Efficient cores, these CPUs deliver excellent gaming and multitasking performance.
   - **AMD's Ryzen 7000 Series**: Utilizing the Zen 4 architecture, these CPUs support DDR5 memory and PCIe 

# 1️⃣ **Grammar Mode with BNF for Computer Specifications**

### **What is Grammar Mode?**
Grammar Mode ensures that the model outputs responses that conform to a strict structure, preventing unexpected variations in responses. This is particularly useful for generating structured data, such as hardware specifications.

#### **How Grammar Mode Works**
- Uses a **GBNF (Grammar-Based Normal Form)** to constrain the model's output.
- Ensures the output adheres to predefined rules for valid computer system specifications.

Read more about [GBNF](https://github.com/ggerganov/llama.cpp/tree/master/grammars), and how to [use it](https://docs.fireworks.ai/structured-responses/structured-output-grammar-based).


In [48]:
computer_system_grammar = """
root ::= system-description

system-description ::= "Processor: " processor "Memory: " memory "Storage: " storage "GPU: " gpu "Motherboard: " motherboard "Cooling: " cooling "Networking: " networking "Ports: " ports "OS: " os

processor ::= cpu-type " " cores " cores @ " clock-speed "GHz\n"
cpu-type ::= ("Intel Core i" ("5" | "7" | "9") | "AMD Ryzen " ("5" | "7" | "9"))
cores ::= ("4" | "6" | "8" | "12" | "16")
clock-speed ::= ("2.5" | "3.0" | "3.5" | "4.0" | "4.5")

memory ::= ram-size "GB " ram-type "RAM \n"
ram-size ::= ("8" | "16" | "32" | "64" | "128")
ram-type ::= ("DDR4" | "DDR5")

storage ::= storage-type " " storage-capacity "TB\n"
storage-type ::= ("SSD" | "HDD" | "NVMe SSD")
storage-capacity ::= ("0.5" | "1" | "2" | "4" | "8")

gpu ::= gpu-brand " " gpu-model "\n"
gpu-brand ::= ("NVIDIA" | "AMD")
gpu-model ::= ("RTX " ("3060" | "3070" | "3080" | "3090") | "RX " ("6600" | "6700" | "6800" | "6900"))

motherboard ::= ("ATX" | "Micro-ATX" | "Mini-ITX")
cooling ::= ("Air-cooled" | "Liquid-cooled")
networking ::= ("WiFi 6" | "WiFi 6E" | "Gigabit Ethernet" | "2.5G Ethernet")
ports ::= ("USB-C, HDMI, Thunderbolt 4" | "USB-A, DisplayPort, Ethernet" | "USB 3.0, VGA, HDMI")

os ::= operating-system "\n"
operating-system ::= ("Windows 11" | "macOS Sonoma" | "Ubuntu 22.04" | "Fedora 38")
"""

In [49]:
chat_completion = client.chat.completions.create(
    model=model_name,
    response_format={"type": "grammar", "grammar": computer_system_grammar},
    messages=[
        {"role": "system", "content": "Generate a structured computer system specification."},
        {"role": "user", "content": "I need a high-performance gaming PC with lots of RAM and a powerful NVIDIA GPU."},
    ],
)
print(chat_completion.choices[0].message.content)

Processor: Intel Core i9 8 cores @ 4.5GHz
Memory: 32GB DDR5RAM 
Storage: SSD 1TB
GPU: NVIDIA RTX 3080
Motherboard: ATXCooling: Liquid-cooledNetworking: WiFi 6EPorts: USB-A, DisplayPort, EthernetOS: Windows 11



# 2️⃣ **JSON Mode for Computer System Specifications**

JSON mode is a subset of grammar mode and is a feature that allows you to specify a JSON schema to force a language model to respond in a structured JSON format. This has several benefits:

* Clarity and precision in responses
* Easy parsing and processing of output
* Improved integration with other systems and applications
* Standardized format for consistent data handling

By using JSON mode, developers can ensure more predictable and easily manageable outputs from AI models, making it easier to incorporate AI capabilities into existing workflows and applications.

In [50]:
from pydantic import BaseModel, Field
from typing import List

In [51]:
class Processor(BaseModel):
    brand: str = Field(pattern="^(Intel|AMD)$")
    model: str
    cores: int = Field(ge=2, le=32)
    clock_speed_ghz: float = Field(ge=1.0, le=6.0)

class Memory(BaseModel):
    size_gb: int = Field(ge=4, le=256)
    type: str = Field(pattern="^(DDR4|DDR5)$")

class Storage(BaseModel):
    type: str = Field(pattern="^(SSD|HDD|NVMe SSD)$")
    capacity_tb: float = Field(ge=0.5, le=10)

class GPU(BaseModel):
    brand: str = Field(pattern="^(NVIDIA|AMD)$")
    model: str

class ComputerSystemSpecs(BaseModel):
    processor: Processor
    memory: Memory
    storage: Storage
    gpu: GPU
    motherboard: str = Field(pattern="^(ATX|Micro-ATX|Mini-ITX)$")
    cooling: str = Field(pattern="^(Air-cooled|Liquid-cooled)$")
    networking: str = Field(pattern="^(WiFi 6|WiFi 6E|Gigabit Ethernet|2.5G Ethernet)$")
    ports: List[str] = Field(min_items=1, max_items=5)
    os: str = Field(pattern="^(Windows 11|macOS Sonoma|Ubuntu 22.04|Fedora 38)$")


In [52]:
chat_completion = client.chat.completions.create(
    model=model_name,
    response_format={"type": "json_object", "schema": ComputerSystemSpecs.model_json_schema()},
    messages=[
        {"role": "user", "content": "Generate a high-performance gaming PC specification with the latest AMD processor, 32GB DDR5 RAM, and a powerful NVIDIA GPU."},
    ]
)
print(chat_completion.choices[0].message.content)

{"processor": {"brand": "AMD", "model": "Ryzen 9 7950X", "cores": 16, "clock_speed_ghz": 4.5}, "memory": {"size_gb": 32, "type": "DDR5"}, "storage": {"type": "NVMe SSD", "capacity_tb": 2}, "gpu": {"brand": "NVIDIA", "model": "GeForce RTX 4090"}, "motherboard": "ASUS ROG Crosshair X670E Hero", "cooling": "NZXT Kraken X73 360mm", "networking": "Intel Wi-Fi 6E AX210", "ports": [ "USB Type-C", "HDMI", "DisplayPort", "Ethernet"], "os": "Windows 11 Pro"}


# 3️⃣ **Reasoning Model JSON Mode for Computer Specifications**

### 🧠 **Reasoning Model JSON Mode for Computer Specifications**  

#### 🔹 **What is Reasoning JSON Mode?**  
Reasoning JSON Mode extends structured JSON generation by adding an **explanation of the model’s decision-making process**. This is done by generating a **reasoning section enclosed within `<think>...</think>` tags**, followed by a structured JSON response.  

This approach enhances transparency and interpretability by providing insight into **why** the model chose specific components when generating PC specifications.

---

#### 🔹 **How Does It Work?**  
When using Reasoning JSON Mode, the model outputs:  
1. A **reasoning block** inside `<think>...</think>`, explaining the thought process.  
2. A **valid JSON object** containing structured specifications that adhere to the predefined schema.  

This ensures that the **reasoning and structured response remain distinct**, making it easy to extract insights while maintaining machine-readable outputs.

---

#### 🔹 **Why Use Reasoning JSON Mode for PC Specifications?**  
For tasks like generating **gaming PC configurations**, **server builds**, or **custom hardware recommendations**, Reasoning JSON Mode provides:  
✅ **Explainability** → Understand why each component was selected.  
✅ **Debugging Capability** → Identify model errors by checking its reasoning.  
✅ **Structured Data** → JSON output is ready for downstream processing.  

For example, instead of simply outputting:
```json
{ "processor": "AMD Ryzen 9 7950X", "gpu": "NVIDIA RTX 3090" }
```
The model also includes:

```
The user requested a high-performance gaming PC, so I chose the Ryzen 9 7950X for its high core count and strong gaming performance. The RTX 3090 is selected because it balances price and power for 4K gaming.
</think>
```
This helps developers validate the AI's decision-making while maintaining structured responses.

#### 🔹 **How is Reasoning JSON Mode Different from Standard JSON Mode?**  
| Feature  | Standard JSON Mode  | Reasoning JSON Mode |
|----------|--------------------|---------------------|
| **Structured Output** | ✅ Yes | ✅ Yes |
| **Model Explainability** | ❌ No | ✅ Yes |
| **Debugging Insights** | ❌ No | ✅ Yes |
| **Separation of Reasoning & JSON** | ❌ No | ✅ Yes |

Reasoning JSON Mode is particularly useful when **generating recommendations, debugging AI outputs, or ensuring trust in AI-driven decisions**.



In [53]:
import json
import re
from pydantic import BaseModel
import os

In [54]:
# Define the output schema using Pydantic
class ComputerSpec(BaseModel):
    processor: str
    memory: str
    storage: str
    gpu: str
    motherboard: str
    cooling: str
    networking: str
    ports: str
    os: str

In [55]:
# Prepare the user input
user_input = "Generate a high-performance gaming PC specification with an AMD Ryzen 9 processor, 64GB DDR5 RAM, and an NVIDIA RTX 3090 GPU."

# Construct the messages payload
messages = [{"role": "user", "content": user_input}]

print(messages)

[{'role': 'user', 'content': 'Generate a high-performance gaming PC specification with an AMD Ryzen 9 processor, 64GB DDR5 RAM, and an NVIDIA RTX 3090 GPU.'}]


In [56]:
# Make the API call to the model
response = client.chat.completions.create(
    model="accounts/fireworks/models/deepseek-r1",
    messages=messages,
    response_format={"type": "json_object", "schema": ComputerSpec.model_json_schema()},
    max_tokens=3000,
)

# Extract the content of the response
response_content = response.choices[0].message.content
print(response_content)


<think>
Okay, I need to create a high-performance gaming PC build with an AMD Ryzen 9 processor, 64GB DDR5 RAM, and an NVIDIA RTX 3090 GPU. Let me start by breaking down each component. 

First, the AMD Ryzen 9. The latest generation for Ryzen processors is the 7000 series, so the Ryzen 9 7950X seems like a good choice. It's a 16-core, 32-thread CPU with high clock speeds, which is great for gaming and multitasking. But wait, do gamers really need 16 cores? Maybe a Ryzen 9 7900X (12-core) is more than enough. However, since the user specified "high-performance," the 7950X might be better to showcase top-tier specs.

Next, the motherboard. It needs to support DDR5 and have an AM5 socket for the Ryzen 7000 series. X670E chipsets are high-end and offer PCIe 5.0 support. Brands like ASUS, MSI, or Gigabyte. The ASUS ROG Crosshair X670E Hero comes to mind. It has good VRM cooling, multiple M.2 slots, and robust build quality. It's a bit pricey, but suitable for a high-end build.

RAM: 64GB o

In [57]:
# Extract the reasoning part enclosed in <think>...</think> tags
reasoning_match = re.search(r"<think>(.*?)</think>", response_content, re.DOTALL)
reasoning = reasoning_match.group(1).strip() if reasoning_match else "No reasoning provided."


In [58]:
# Extract the JSON part that follows after the reasoning
json_match = re.search(r"</think>\s*(\{.*\})", response_content, re.DOTALL)
json_str = json_match.group(1).strip() if json_match else "{}"

print(json_str)

{"processor":"AMD Ryzen 9 7950X 4.5GHz 16-Core (Boost: 5.7GHz)", "memory":"64GB G.Skill Trident Z5 RGB DDR5-6000 CL36 (2x32GB)", "storage":"Primary: 2TB Samsung 990 Pro NVMe PCIe 4.0 | Secondary: 4TB Crucial P3 Plus NVMe PCIe 4.0", "gpu":"ASUS ROG Strix GeForce RTX 3090 OC 24GB GDDR6X", "motherboard":"ASUS ROG Crosshair X670E Hero (Wi-Fi 6E, PCIe 5.0)", "cooling":"NZXT Kraken Z73 360mm AIO Liquid Cooler", "networking":"Motherboard-integrated Wi-Fi 6E | 10G LAN", "ports":"USB4 (Type-C), 12x USB 3.2, HDMI 2.1", "os":"Windows 11 Home 64-bit (optimized for DirectStorage and gaming)"}


In [59]:
# Parse the JSON string directly into a Pydantic model using model_validate_json
computer_spec = ComputerSpec.model_validate_json(json_str)
print(computer_spec)

processor='AMD Ryzen 9 7950X 4.5GHz 16-Core (Boost: 5.7GHz)' memory='64GB G.Skill Trident Z5 RGB DDR5-6000 CL36 (2x32GB)' storage='Primary: 2TB Samsung 990 Pro NVMe PCIe 4.0 | Secondary: 4TB Crucial P3 Plus NVMe PCIe 4.0' gpu='ASUS ROG Strix GeForce RTX 3090 OC 24GB GDDR6X' motherboard='ASUS ROG Crosshair X670E Hero (Wi-Fi 6E, PCIe 5.0)' cooling='NZXT Kraken Z73 360mm AIO Liquid Cooler' networking='Motherboard-integrated Wi-Fi 6E | 10G LAN' ports='USB4 (Type-C), 12x USB 3.2, HDMI 2.1' os='Windows 11 Home 64-bit (optimized for DirectStorage and gaming)'


In [60]:
# Output the reasoning and the parsed JSON data
print("Reasoning:")
print(reasoning)


Reasoning:
Okay, I need to create a high-performance gaming PC build with an AMD Ryzen 9 processor, 64GB DDR5 RAM, and an NVIDIA RTX 3090 GPU. Let me start by breaking down each component. 

First, the AMD Ryzen 9. The latest generation for Ryzen processors is the 7000 series, so the Ryzen 9 7950X seems like a good choice. It's a 16-core, 32-thread CPU with high clock speeds, which is great for gaming and multitasking. But wait, do gamers really need 16 cores? Maybe a Ryzen 9 7900X (12-core) is more than enough. However, since the user specified "high-performance," the 7950X might be better to showcase top-tier specs.

Next, the motherboard. It needs to support DDR5 and have an AM5 socket for the Ryzen 7000 series. X670E chipsets are high-end and offer PCIe 5.0 support. Brands like ASUS, MSI, or Gigabyte. The ASUS ROG Crosshair X670E Hero comes to mind. It has good VRM cooling, multiple M.2 slots, and robust build quality. It's a bit pricey, but suitable for a high-end build.

RAM: 64G

In [61]:
print("\nComputer Specification (JSON Data):")
print(computer_spec.model_dump_json(indent=4))


Computer Specification (JSON Data):
{
    "processor": "AMD Ryzen 9 7950X 4.5GHz 16-Core (Boost: 5.7GHz)",
    "memory": "64GB G.Skill Trident Z5 RGB DDR5-6000 CL36 (2x32GB)",
    "storage": "Primary: 2TB Samsung 990 Pro NVMe PCIe 4.0 | Secondary: 4TB Crucial P3 Plus NVMe PCIe 4.0",
    "gpu": "ASUS ROG Strix GeForce RTX 3090 OC 24GB GDDR6X",
    "motherboard": "ASUS ROG Crosshair X670E Hero (Wi-Fi 6E, PCIe 5.0)",
    "cooling": "NZXT Kraken Z73 360mm AIO Liquid Cooler",
    "networking": "Motherboard-integrated Wi-Fi 6E | 10G LAN",
    "ports": "USB4 (Type-C), 12x USB 3.2, HDMI 2.1",
    "os": "Windows 11 Home 64-bit (optimized for DirectStorage and gaming)"
}


# **Conclusion**



To learn more, visit [Fireworks AI Docs](https://docs.fireworks.ai) or join our [Discord community](https://discord.gg/fireworks-ai).